In [ ]:
using ModiaMath
using StaticArrays

@component Pendulum(;L=1.0, m=1.0, d=0.1, g=9.81) begin
 phi = RealScalar(start=pi/2, unit="rad"    , fixed=true,               numericType=ModiaMath.XD_EXP)
 w   = RealScalar(start=0.0 , unit="rad/s"  , fixed=true, integral=phi, numericType=ModiaMath.XD_EXP)
 a   = RealScalar(            unit="rad/s^2",             integral=w  , numericType=ModiaMath.DER_XD_EXP) 
 r   = RealSVector{2}(        unit="m"      ,                           numericType=ModiaMath.WC)
end;

function ModiaMath.computeVariables!(p::Pendulum, sim::ModiaMath.SimulationState)  
 L = p.L; m = p.m; d = p.d; g = p.g; phi = p.phi.value; w = p.w.value

 p.a.value = (-m*g*L*sin(phi) - d*w) / (m*L^2)

 if ModiaMath.isStoreResult(sim)
    p.r.value = @SVector [L*sin(phi), -L*cos(phi)]
 end
end;

simulationModel = ModiaMath.SimulationModel(Pendulum(L=0.8, m=0.5, d=0.2), stopTime=5.0);

In [2]:
result = ModiaMath.simulate!(simulationModel; log=true);

... ModiaMath.simulate! (version 0.5.1 2019-04-07) to simulate model: Pendulum
      Initialization at time = 0.0 s
        initial values:
          │ x │ name   │ start   │ fixed │ nominal │
          ├───┼────────┼─────────┼───────┼─────────┤
          │ 1 │ phi    │ 1.5708  │ true  │ 1.5708  │
          │ 2 │ w      │ 0.0     │ true  │ 1.0     │

        for given x, determine consistent DAE variables der(x) (solving a linear equation system)
      Simulation started

      Simulation is terminated at time = 5.0 s


        structureOfDAE = DAE_LinearDerivativesAndConstraints
        cpuTime        = 2.6 s (init: 2 s, integration: 0.6 s)
        startTime      = 0.0 s
        stopTime       = 5.0 s
        interval       = 0.01 s
        tolerance      = 0.0001
        nEquations     = 2 (includes 0 constraints)
        nResults       = 501
        nSteps         = 137
        nResidues      = 218 (includes residue calls for Jacobian)
        nZeroCrossings = 0
        nJac           = 21
        nTimeEvents    = 0
        nStateEvents   = 0
        nRestartEvents = 0
        nErrTestFails  = 2
        h0             = 5.8e-07 s
        hMin           = 5.8e-07 s
        hMax           = 0.081 s
        orderMax       = 5
        sparseSolver   = false



      Statistics (get help with ?ModiaMath.SimulationStatistics):


In [ ]:
ModiaMath.plot(result, [(:phi, :w) :a])